In [1]:
# imports for example data and model training
from credoai.data import fetch_creditdefault
from sklearn.ensemble import GradientBoostingClassifier

# Base Lens imports
import credoai.lens as cl



Failed to import tensorflow. Please note that tensorflow is not installed by default when you install tensorflow_hub. This is so that users can decide which tensorflow package to use. To use tensorflow_hub, please install a current version of tensorflow by following the instructions at https://tensorflow.org/install and https://tensorflow.org/hub/installation.




In [2]:
data = fetch_creditdefault(as_frame=True)
X = data['data'].drop(columns=['SEX'])
y = data['target']
sensitive_feature = data['data']['SEX']

In [3]:
model = GradientBoostingClassifier()
model.fit(X,y)

GradientBoostingClassifier()

In [4]:
credo_model = cl.CredoModel(name='credit_default_classifier',
                            model=model)

credo_data = cl.CredoData(name='UCI-credit-default',
                          X=X, 
                          y=y.astype(int),
                          sensitive_features=sensitive_feature)
credo_governance = cl.CredoGovernance(ai_solution_id=None,
                                      model_id="v3saQTs9UnvVBz9Y9k5Lo2"
                                     )

In [5]:
# specify the metrics that will be used by the FairnessBase assessment
alignment_spec = {
    'FairnessBase': {'metrics': ['precision_score']}
}

#### Run Lens

Once we have the model and data artifacts, as well as the spec, we can run Lens. By default it will automatically infer which assessments to run, just as we manually did above!

In [6]:
from credoai.assessment import FairnessBaseAssessment
lens = cl.Lens(model=credo_model,
               governance=credo_governance,
               assessments = [FairnessBaseAssessment()],
               data=credo_data,
               spec=alignment_spec)

In [7]:
lens.run_assessments(export=True)

{'FairnessBase': {'fairness':                     value
  precision_score  0.000543,
  'disaggregated_results':          precision_score
  SEX                     
  1.0             0.695954
  2.0             0.695411
  overall         0.695652}}

In [ ]:
m = lens.assessments['FairnessBase']

In [14]:
self = lens
export=True
assessment_kwargs =  {}
assessment_results = {}
for name, assessment in self.assessments.items():
    kwargs = assessment_kwargs.get(name, {})
    assessment_results[name] = assessment.run(**kwargs)
    if export:
        prepared_results = self._prepare_results(assessment, **kwargs)
        if type(export) == str:
            self._export_results_to_file(prepared_results, export)
        else:
            self._export_results_to_credo(prepared_results, to_model=True)

In [16]:
import credoai.integration as ci
results = prepared_results
to_model = True
metric_records = ci.record_metrics(results)
destination_id = self.gov.model_id if to_model else self.gov.data_id
ci.export_to_credo(metric_records, destination_id)

In [45]:
model_record = metric_records

In [46]:
model_record.credoify()

'{"data": [{"attributes": {"type": "precision_score", "value": 0.0005434945456098683, "dataset_variant_id": null, "metadata": {"creation_time": "14-01-2022 16:56:52", "model_label": "credit_default_classifier", "dataset_label": "UCI-credit-default", "user_id": null, "assessment": "FairnessBase", "lens_version": "Lens_v0.0.2"}}, "type": "model_metrics"}, {"attributes": {"type": "precision_score_SEX-1.0", "value": 0.6959544879898862, "dataset_variant_id": null, "metadata": {"creation_time": "14-01-2022 16:56:52", "model_label": "credit_default_classifier", "dataset_label": "UCI-credit-default", "user_id": null, "assessment": "FairnessBase", "lens_version": "Lens_v0.0.2"}}, "type": "model_metrics"}, {"attributes": {"type": "precision_score_SEX-2.0", "value": 0.6954109934442764, "dataset_variant_id": null, "metadata": {"creation_time": "14-01-2022 16:56:52", "model_label": "credit_default_classifier", "dataset_label": "UCI-credit-default", "user_id": null, "assessment": "FairnessBase", "le

In [42]:
from credoai.utils.credo_api_utils import *
def patch_metrics(model_id, model_record):
    """Send a model record object to Credo's Governance Platform
    
    Parameters
    ----------
    model_id : string
        Identifier for Model on Credo AI Governance Platform
    model_record : Record
        Model Record object, see credo.integration.MutliRecord
    """
    end_point = get_end_point(f"models/{model_id}/relationships/metrics")
    return submit_request('patch', end_point, data=model_record, headers={"content-type": "application/vnd.api+json"})

In [28]:
a=patch_metrics(credo_governance.model_id, model_record)

https://api.credo-qa.com/api/v1/credoai/models/v3saQTs9UnvVBz9Y9k5Lo2/relationships/metrics


In [41]:
from json_api_doc import serialize
a=serialize(data=model_record._struct())
a['included'] = []
out = json.dumps(a)

In [44]:
patch_metrics(credo_governance.model_id,out)

<Response [200]>

In [33]:
import pprint
import json
pprint.pprint(json.loads(model_record.credoify()))

{'data': [{'attributes': {'dataset_variant_id': None,
                          'metadata': {'assessment': 'FairnessBase',
                                       'creation_time': '14-01-2022 16:56:52',
                                       'dataset_label': 'UCI-credit-default',
                                       'lens_version': 'Lens_v0.0.2',
                                       'model_label': 'credit_default_classifier',
                                       'user_id': None},
                          'type': 'precision_score',
                          'value': 0.0005434945456098683},
           'type': 'model_metrics'},
          {'attributes': {'dataset_variant_id': None,
                          'metadata': {'assessment': 'FairnessBase',
                                       'creation_time': '14-01-2022 16:56:52',
                                       'dataset_label': 'UCI-credit-default',
                                       'lens_version': 'Lens_v0.0.2',
           